In [9]:
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [59]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")



llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-05-20",
    
)



messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime la programmation.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-05-20', 'safety_ratings': []}, id='run--ddda1cc1-9cb8-4868-8008-8f6ef085a537-0', usage_metadata={'input_tokens': 21, 'output_tokens': 7, 'total_tokens': 111, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 83}})

In [61]:
smoll_llm = ChatOpenAI(
    model='deepseek/deepseek-r1-0528-qwen3-8b:free',
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ['OPEN_ROUTER_KEY'],
    temperature=0.15
)


smoll_llm.invoke(messages)

AIMessage(content="J'aime le programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 274, 'prompt_tokens': 25, 'total_tokens': 299, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1-0528-qwen3-8b:free', 'system_fingerprint': None, 'id': 'gen-1749389531-t24c4fDMB6e6rlKDrvBW', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--8c0e02b6-1c1c-4042-ab6b-b48c7f098d6f-0', usage_metadata={'input_tokens': 25, 'output_tokens': 274, 'total_tokens': 299, 'input_token_details': {}, 'output_token_details': {}})

## Fake Data Generation

In [67]:
from datetime import date
from enum import Enum
from typing import List
from pydantic import BaseModel, Field


class Sex(str, Enum):
    M = 'M'
    F = 'F'
    O = 'O'


class EntryType(str, Enum):
    weight_check = 'weight_check'
    meal         = 'meal'
    workout      = 'workout'
    note         = 'note'


class UserSample(BaseModel):
    email: str = Field(..., description="User's email address, must be unique")
    password: str = Field(..., description="password for authentication")
    first_name: str = Field(None, description="User's given name")
    last_name: str = Field(None, description="User's family name")
    date_of_birth: date = Field(None, description="Date of birth for age calculation")
    sex: Sex = Field(None, description="Sex of the user: M, F, or O (other)")
    height_cm: float = Field(None, description="Height in centimeters")
    weight_kg: float = Field(None, description="Weight in kilograms")
    activity_level: str = Field(None, description="Free-form description of daily activity level")
    dietary_pref: str = Field(None, description="Free-form dietary preference or restriction")
    fitness_goals: str = Field(None, description="Free-form fitness goals the user wants to achieve")
    


class UserSamples(BaseModel):
    users: List[UserSample] = Field(description="List of user samples")

In [69]:
DATA_GENERATION_PROMPT = """
You are an expert on generating fake user and fitness data.

# Instruction
- Generate information of a fake user of a fitness app.
- Pay attention to fields descriptions for generating valid data.
- Generate {num_samples} samples
"""

In [72]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import JsonOutputParser

sample_num = 10

messages = [
    HumanMessage(
        content = DATA_GENERATION_PROMPT.format(num_samples = sample_num)
    )
]

sampler = smoll_llm.with_structured_output(UserSamples)

In [73]:
samples = sampler.invoke(messages)
samples

UserSamples(users=[UserSample(email='user1@example.com', password='password123', first_name='John', last_name='Doe', date_of_birth=datetime.date(1985, 5, 15), sex=<Sex.M: 'M'>, height_cm=180.0, weight_kg=85.0, activity_level='moderate', dietary_pref='vegetarian', fitness_goals='weight_loss'), UserSample(email='user2@example.com', password='password456', first_name='Jane', last_name='Smith', date_of_birth=datetime.date(1990, 8, 22), sex=<Sex.F: 'F'>, height_cm=165.0, weight_kg=60.0, activity_level='sedentary', dietary_pref='gluten_free', fitness_goals='muscle_gain'), UserSample(email='user3@example.com', password='password789', first_name='Michael', last_name='Johnson', date_of_birth=datetime.date(1978, 11, 30), sex=<Sex.M: 'M'>, height_cm=175.0, weight_kg=90.0, activity_level='active', dietary_pref='none', fitness_goals='endurance'), UserSample(email='user4@example.com', password='password101', first_name='Emily', last_name='Williams', date_of_birth=datetime.date(1992, 3, 14), sex=<Sex

In [76]:
from db.db_manager import FitnessDB

fittness_db = FitnessDB()

In [84]:
from werkzeug.security import generate_password_hash, check_password_hash

for user in samples.users:
    fittness_db.create_user(
        email = user.email,
        password_hash = generate_password_hash(user.password),
        first_name = user.first_name,
        last_name = user.last_name,
        date_of_birth = user.date_of_birth,
        sex =  user.sex.value,
        height_cm = user.height_cm,
        weight_kg = user.weight_kg,
        activity_level = user.activity_level,
        dietary_pref = user.dietary_pref,
        fitness_goals = user.fitness_goals,
    )

In [86]:
import json

with open("data/users.json", 'w') as f:
    json.dump(samples.model_dump_json(), f)

In [ ]:
class Graph:
    
    def __init__(self, llm, message_class):
        self.llm = llm
        self.message_class = message_class
        
    def compile(self):
        raise NotImplementedError()

In [21]:
from langgraph.graph import MessagesState
from pydantic import BaseModel, Field
from typing import List, Literal

class UserMessages(MessagesState):
    user_id: int

In [89]:
FITNESS_SYSTEM_PROMPT = """
You are a smart, supportive diet & fitness AI assistant. Your job is to craft personalized, goal‐driven workout plans based on each user’s profile.

# Instructions
1. Structure the plan into three phases:  
   • Warm-up (5–10 minutes)  
   • Main workout (strength/cardio/core, etc.)  
   • Cool-down & stretching  

2. Tailor every exercise choice to the user’s stated goals, experience level, and any preferences or limitations.

3. For each exercise, include:  
   • Clear instructions on form & reps/sets/duration  
   • Key benefits (e.g. “Strengthens glutes and improves hip stability”)  

4. Make the plan actionable and detailed—so a user can open it and start immediately without guesswork.

Keep the tone encouraging and professional.  
# User Profile
Here is the user info:
{user_info}
"""


In [94]:
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import END, START, StateGraph

class PlannerGraph(Graph):
    
    def __init__(self, llm, db_manager: FitnessDB, planner_sys_prompt: str):
        super().__init__(llm, UserMessages)
        self.db_manager = db_manager
        self.planner_sys_prompt = planner_sys_prompt
    
    
    def node_plan(self, state):
        
        user_info = self.db_manager.get_user(state['user_id'])
        messages = [
            SystemMessage(
                content = self.planner_sys_prompt.format(user_info = user_info)
            )
        ]
        response = llm.invoke(messages + state['messages'])
        state['messages'].append(response)
        return response

    
    def compile(self):
        graph_builder = StateGraph(self.message_class)
        graph_builder.add_node("node_plan", self.node_plan)
        graph_builder.add_edge(START, "node_plan")
        graph_builder.add_edge("node_plan", END)
        self.graph_compiled = graph_builder.compile()
        return self.graph_compiled
        
        
        
        

In [95]:
fitnessgraph = PlannerGraph(llm, fittness_db, FITNESS_SYSTEM_PROMPT).compile()

In [96]:
messages = [HumanMessage(content = 'give me a plan.')]
fitnessgraph.invoke({
    'messages':  messages,
    'user_id': 1
})

{'messages': [HumanMessage(content='give me a plan.', additional_kwargs={}, response_metadata={}, id='e65b60c6-e60b-4424-9e7e-af7a494214d5'),
  AIMessage(content="Here is a personalized workout plan designed to help you achieve your weight loss goals, tailored to your moderate activity level. This plan focuses on a full-body approach, combining strength and cardiovascular elements to maximize calorie burn and build lean muscle.\n\nLet's get you started, John!\n\n---\n\n### **Your Personalized Weight Loss Workout Plan**\n\n**Goal:** Weight Loss\n**Focus:** Full-body strength, cardiovascular endurance, and calorie expenditure.\n\n**Instructions:**\n*   Aim to perform this workout 3-4 times per week, with at least one rest day in between sessions.\n*   Listen to your body. If an exercise causes pain, stop immediately.\n*   Focus on proper form over speed or number of reps.\n*   Stay hydrated throughout your workout.\n\n---\n\n#### **Phase 1: Warm-up (8-10 minutes)**\nThis phase prepares y